In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostRegressor
import clarke_error_grid_analysis as cega
from sklearn.preprocessing import MinMaxScaler


In [20]:
scaler = MinMaxScaler()

In [21]:
def get_data(file_path):
    data = pd.read_csv(file_path)
    
    #  k cross validation
    #  Shuffle the data
    data = data.sample(frac=1).reset_index(drop=True)
    
    # Drop columns with missing values
    data = data.dropna(axis=1)
    
    # drop columns with string values
    data = data.select_dtypes(exclude=['object'])
    
    # drop columns with infinite values
    data = data.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    
    # normalize the data
    
    # data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
    
    # Last column is the target
    y = data.iloc[:, -1]
    X = data.iloc[:, :-1]
    
    cols = ['Tpi_mean', 'Tpi_median', 'Tpi_std', 
            'Tpp_mean', 'Tpp_median', 'Tpp_std',
            'Tsys_mean', 'Tsys_median', 'Tsys_std',
            'Tdia_mean', 'Tdia_median', 'Tdia_std',
            'Tsp_mean', 'Tsp_median', 'Tsp_std',
            'Tdp_mean', 'Tdp_median', 'Tdp_std',
            'deltaT_mean', 'deltaT_median', 'deltaT_std',
            'Asp_mean', 'Asp_median', 'Asp_std',
            'Adp_mean', 'Adp_median', 'Adp_std',
            'Adn_mean', 'Adn_median', 'Adn_std',
            'Aoff_mean', 'Aoff_median', 'Aoff_std',
            'AUCpi_mean', 'AUCpi_median', 'AUCpi_std',
            'AUCdia_mean', 'AUCdia_median', 'AUCdia_std',
            'AUCsys_mean', 'AUCsys_median', 'AUCsys_std',
            'IPR_mean', 'IPR_median', 'IPR_std','ppg_p_1',
            'ppg_f_1','ppg_p_2','ppg_f_2','ppg_p_3','ppg_f_3','ppg_pow','ppg_rpow',
            'Tsys_iqr', 'Tdp_std', 'Tdp_iqr', 'Tdw75_mean', 'Tdw90_median']
    

    
#     X = X[cols]
    
    
    X = X.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    

    # Split the data into training and temporary sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.7, random_state=0)

    # Further split the temporary set into validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

    # Convert y_train, y_val, and y_test to NumPy arrays
    y_train = y_train.to_numpy()
    y_val = y_val.to_numpy()
    y_test = y_test.to_numpy()

    return X_train, X_val, X_test, y_train, y_val, y_test


In [22]:
def polynomial_regression(X_train, y_train, X_test, y_test, X_val, y_val):
    # Polynomial regression
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.pipeline import make_pipeline

    # Create a pipeline
    model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())

    # Fit the model
    model.fit(X_train, y_train)

    # Predict the target values
    y_pred = model.predict(X_test)

    y_pred_val = model.predict(X_val)

    
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_test = r2_score(y_test, y_pred)
    mae_test = np.mean(np.abs(y_test - y_pred))
    mse_test = mean_squared_error(y_test, y_pred)
    clark_test = cega.zone_accuracy(y_test, y_pred)
    
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2_val = r2_score(y_val, y_pred_val)
    mae_val = np.mean(np.abs(y_val - y_pred_val))
    mse_val = mean_squared_error(y_val, y_pred_val)
    clark_val = cega.zone_accuracy(y_val, y_pred_val)
    
    return ["pol reg", rmse_test, r2_test, mae_test, mse_test, clark_test, rmse_val, r2_val, mae_val, mse_val, clark_val]

In [23]:
def linear_regression(X_train, y_train, X_test, y_test, X_val, y_val):
    # Create a linear regression model
    model = LinearRegression()
    
    # normalize y
    y_train = np.log(y_train)
    y_test = np.log(y_test)
    y_val = np.log(y_val)
    

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_val = model.predict(X_val)
    
    # denormalize y
    y_train = np.exp(y_train)
    y_test = np.exp(y_test)
    y_val = np.exp(y_val)
    
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_test = r2_score(y_test, y_pred)
    mae_test = np.mean(np.abs(y_test - y_pred))
    mse_test = mean_squared_error(y_test, y_pred)
    clark_test = cega.zone_accuracy(y_test, y_pred)
    
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2_val = r2_score(y_val, y_pred_val)
    mae_val = np.mean(np.abs(y_val - y_pred_val))
    mse_val = mean_squared_error(y_val, y_pred_val)
    clark_val = cega.zone_accuracy(y_val, y_pred_val)
    
    
    return ["lin reg", rmse_test, r2_test, mae_test, mse_test, clark_test, rmse_val, r2_val, mae_val, mse_val, clark_val]

In [24]:
def xgboost(X_train, y_train, X_test, y_test, X_val, y_val):
    # xg boost
    model = XGBRegressor()
    model.fit(X_train, y_train)
    
    # normalize y
    y_train = np.log(y_train)
    y_test = np.log(y_test)
    y_val = np.log(y_val)
    
    y_pred = model.predict(X_test)
    y_pred_val = model.predict(X_val)
    
    # denormalize y
    y_train = np.exp(y_train)
    y_test = np.exp(y_test)
    y_val = np.exp(y_val)
    
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_test = r2_score(y_test, y_pred)
    mae_test = np.mean(np.abs(y_test - y_pred))
    mse_test = mean_squared_error(y_test, y_pred)
    clark_test = cega.zone_accuracy(y_test, y_pred)
    
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2_val = r2_score(y_val, y_pred_val)
    mae_val = np.mean(np.abs(y_val - y_pred_val))
    mse_val = mean_squared_error(y_val, y_pred_val)
    clark_val = cega.zone_accuracy(y_val, y_pred_val)
    
    
    return ["xgboost", rmse_test, r2_test, mae_test, mse_test, clark_test, rmse_val, r2_val, mae_val, mse_val, clark_val]    
    

In [25]:
def decision_tree(X_train, y_train, X_test, y_test, X_val, y_val):
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)
    
    # normalize y
    y_train = np.log(y_train)
    y_test = np.log(y_test)
    y_val = np.log(y_val)
    
    y_pred = model.predict(X_test)
    y_pred_val = model.predict(X_val)
    
    # denormalize y
    y_train = np.exp(y_train)
    y_test = np.exp(y_test)
    y_val = np.exp(y_val)
    
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_test = r2_score(y_test, y_pred)
    mae_test = np.mean(np.abs(y_test - y_pred))
    mse_test = mean_squared_error(y_test, y_pred)
    clark_test = cega.zone_accuracy(y_test, y_pred)
    
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2_val = r2_score(y_val, y_pred_val)
    mae_val = np.mean(np.abs(y_val - y_pred_val))
    mse_val = mean_squared_error(y_val, y_pred_val)
    clark_val = cega.zone_accuracy(y_val, y_pred_val)
    
    return ["decision tree", rmse_test, r2_test, mae_test, mse_test, clark_test, rmse_val, r2_val, mae_val, mse_val, clark_val]

In [26]:
def random_forest(X_train, y_train, X_test, y_test, X_val, y_val):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    # normalize y
    y_train = np.log(y_train)
    y_test = np.log(y_test)
    y_val = np.log(y_val)
    
    y_pred = model.predict(X_test)
    y_pred_val = model.predict(X_val)
    
    # denormalize y
    y_train = np.exp(y_train)
    y_test = np.exp(y_test)
    y_val = np.exp(y_val)
    
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_test = r2_score(y_test, y_pred)
    mae_test = np.mean(np.abs(y_test - y_pred))
    mse_test = mean_squared_error(y_test, y_pred)
    clark_test = cega.zone_accuracy(y_test, y_pred)
    
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2_val = r2_score(y_val, y_pred_val)
    mae_val = np.mean(np.abs(y_val - y_pred_val))
    mse_val = mean_squared_error(y_val, y_pred_val)
    clark_val = cega.zone_accuracy(y_val, y_pred_val)
    
    
    return ["random forest", rmse_test, r2_test, mae_test, mse_test, clark_test, rmse_val, r2_val, mae_val, mse_val, clark_val]

In [27]:
def ada_boost(X_train, y_train, X_test, y_test, X_val, y_val):
    # ada boost
    model = AdaBoostRegressor()
    model.fit(X_train, y_train)
    
    # normalize y
    y_train = np.log(y_train)
    y_test = np.log(y_test)
    y_val = np.log(y_val)
    
    y_pred = model.predict(X_test)
    y_pred_val = model.predict(X_val)
    
    # denormalize y
    y_train = np.exp(y_train)
    y_test = np.exp(y_test)
    y_val = np.exp(y_val)
    
    # inverse transform of y and predictions
    # y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
    # y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))
    
    # y_val = scaler.inverse_transform(y_val.reshape(-1, 1))
    # y_pred_val = scaler.inverse_transform(y_pred_val.reshape(-1, 1))
    
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_test = r2_score(y_test, y_pred)
    mae_test = np.mean(np.abs(y_test - y_pred))
    mse_test = mean_squared_error(y_test, y_pred)
    clark_test = cega.zone_accuracy(y_test, y_pred)
    
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2_val = r2_score(y_val, y_pred_val)
    mae_val = np.mean(np.abs(y_val - y_pred_val))
    mse_val = mean_squared_error(y_val, y_pred_val)
    clark_val = cega.zone_accuracy(y_val, y_pred_val)
    
    print("Ada Boost")
    print(y_pred)
    print(y_test)    
    
    return ["ada boost", rmse_test, r2_test, mae_test, mse_test, clark_test, rmse_val, r2_val, mae_val, mse_val, clark_val]

In [28]:
def mlp(X_train, y_train, X_test, y_test, X_val, y_val):
    # mlp
    
    model = MLPRegressor(hidden_layer_sizes=(300,350), max_iter=1000)
    model.fit(X_train, y_train)
    
    # normalize y
    y_train = np.log(y_train)
    y_test = np.log(y_test)
    y_val = np.log(y_val)
    
    y_pred = model.predict(X_test)
    y_pred_val = model.predict(X_val)
    
    # denormalize y
    y_train = np.exp(y_train)
    y_test = np.exp(y_test)
    y_val = np.exp(y_val)
    
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_test = r2_score(y_test, y_pred)
    mae_test = np.mean(np.abs(y_test - y_pred))
    mse_test = mean_squared_error(y_test, y_pred)
    clark_test = cega.zone_accuracy(y_test, y_pred)
    
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2_val = r2_score(y_val, y_pred_val)
    mae_val = np.mean(np.abs(y_val - y_pred_val))
    mse_val = mean_squared_error(y_val, y_pred_val)
    clark_val = cega.zone_accuracy(y_val, y_pred_val)
    
    
    return ["mlp", rmse_test, r2_test, mae_test, mse_test, clark_test, rmse_val, r2_val, mae_val, mse_val, clark_val]

In [29]:
def run_models(X_train, X_val, X_test, y_train, y_val, y_test):
    res1 = linear_regression(X_train, y_train, X_test, y_test, X_val, y_val)
    res2 = xgboost(X_train, y_train, X_test, y_test, X_val, y_val)
    res3 = decision_tree(X_train, y_train, X_test, y_test, X_val, y_val)
    res4 = random_forest(X_train, y_train, X_test, y_test, X_val, y_val)
    res6 = ada_boost(X_train, y_train, X_test, y_test, X_val, y_val)
    res7 = mlp(X_train, y_train, X_test, y_test, X_val, y_val)
    res8 = polynomial_regression(X_train, y_train, X_test, y_test, X_val, y_val)

    
    return [res1, res2, res3, res4, res6, res7, res8]
    # return [res1]


In [30]:
# select K best features
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression

def select_k_best_features(X_train, X_val, X_test, y_train, k):
    # feature selection
    # select k best features
    
    # mutual information based
    # conditional probabilility 
    # jmim, mrmr, cife, cfs, relief, chi2, f_classif, mutual_info_classif, f_regression, mutual_info_regression
    # jmi feature selection
    # min redundancy max relevence
    
    # if we dont find saturation in elbow plot, go for more features
    
    
    
    fs = SelectKBest(score_func=mutual_info_regression, k=k)
    fs.fit(X_train, y_train)
    
    X_train_fs = fs.transform(X_train)
    X_val_fs = fs.transform(X_val)
    X_test_fs = fs.transform(X_test)
    
    
    return X_train_fs, X_val_fs, X_test_fs

In [31]:
# data1 = pd.read_csv('/Users/sahilsahu/Desktop/folders/Feature-Extraction/code/data/exp3/data_all_1.csv')
# data2 = pd.read_csv('/Users/sahilsahu/Desktop/folders/Feature-Extraction/code/data/exp2/data_all_1.csv')

# combined_data = pd.concat([data1, data2], axis=0)

# combined_data.to_csv('/Users/sahilsahu/Desktop/folders/Feature-Extraction/code/data/exp_2_3_dataset_1.csv', index=False)

In [32]:
datasets = ['/Users/sahilsahu/Desktop/folders/Feature-Extraction/code/data/exp_2_3_dataset_1.csv']

In [33]:
res_df = pd.DataFrame(columns=['dataset', 'k', 'model', 'rmse_test', 'r2_test', 'mae_test', 'mse_test', 'clark_test_A', 'clark_test_B', 'rmse_val', 'r2_val', 'mae_val', 'mse_val', 'clark_val_A', 'clark_val_B'])
features_df = pd.DataFrame(columns=['dataset', 'k', 'features', 'scores'])

k_values = [5,6,7,8,9,10,11,12,13,14,15]


for dataset in datasets:
    for k in k_values:
        X_train, X_val, X_test, y_train, y_val, y_test = get_data(dataset)
        
        fs = SelectKBest(score_func=mutual_info_regression, k=k)
        fs.fit(X_train, y_train)
        
        X_train_fs = fs.transform(X_train)
        X_val_fs = fs.transform(X_val)
        X_test_fs = fs.transform(X_test)
        
        scores = fs.scores_
        selected_scores = scores[fs.get_support(indices=True)]
        
        features_df = features_df.append({'dataset': dataset, 'k': k, 'features': X_test.columns[fs.get_support(indices=True)], 'scores' : selected_scores}, ignore_index=True)
        
        results = run_models(X_train_fs, X_val_fs, X_test_fs, y_train, y_val, y_test)
        # print(results)
        for res in results:
            if res is not None:
                res_df = res_df.append({'dataset': dataset, 'k': k, 'model': res[0], 'rmse_test': res[1], 'r2_test': res[2], 'mae_test': res[3], 'mse_test': res[4], 'clark_test_A': res[5][0], 'clark_test_B' : res[5][1], 'rmse_val': res[6], 'r2_val': res[7], 'mae_val': res[8], 'mse_val': res[9], 'clark_val_A': res[10][0], 'clark_val_B' : res[10][1]}, ignore_index=True)

/var/folders/p0/rxjj0qbs28gc1tq82964w38w0000gp/T/ipykernel_23313/3332540717.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features_df = features_df.append({'dataset': dataset, 'k': k, 'features': X_test.columns[fs.get_support(indices=True)], 'scores' : selected_scores}, ignore_index=True)


Ada Boost
[ 99.25545455  93.18       117.26454545 138.16        86.23888889
  88.24461538  91.53666667  86.85181818 119.09666667 106.33888889
  86.93769231 103.45        90.56090909  91.41571429 114.94
  85.42181818  90.605      117.614      103.45       226.55
 101.72666667 101.72666667  85.42181818  89.605      131.87714286
 100.56666667 119.0825     123.93222222  98.26       119.0825
 117.26454545  90.605       86.44153846 103.25066667 103.19583333
 101.79181818  92.90666667 103.25066667 118.11294118 102.834
  86.93769231 114.94       226.55      ]
[100.34 109.52  97.75  82.64  89.96  82.89 109.14  88.91  89.99  98.6
  95.42 102.03  80.01  81.27 105.39  60.    98.52  83.65  95.58  80.64
 114.92  84.43 105.34  88.    90.92  82.6   81.13 144.85 108.17  74.96
 101.69  92.03  81.78  92.18  89.83 244.96 101.28 101.2   84.92  86.6
 112.42 114.71  99.68]


/var/folders/p0/rxjj0qbs28gc1tq82964w38w0000gp/T/ipykernel_23313/3332540717.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_df = res_df.append({'dataset': dataset, 'k': k, 'model': res[0], 'rmse_test': res[1], 'r2_test': res[2], 'mae_test': res[3], 'mse_test': res[4], 'clark_test_A': res[5][0], 'clark_test_B' : res[5][1], 'rmse_val': res[6], 'r2_val': res[7], 'mae_val': res[8], 'mse_val': res[9], 'clark_val_A': res[10][0], 'clark_val_B' : res[10][1]}, ignore_index=True)
/var/folders/p0/rxjj0qbs28gc1tq82964w38w0000gp/T/ipykernel_23313/3332540717.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_df = res_df.append({'dataset': dataset, 'k': k, 'model': res[0], 'rmse_test': res[1], 'r2_test': res[2], 'mae_test': res[3], 'mse_test': res[4], 'clark_test_A': res[5][0], 'clark_test_B' : res[5][1], 'rmse

Ada Boost
[ 88.11909091  91.0404      94.85148148  92.1425      94.60625
  87.42041667  88.75166667  87.66478261  94.431875    94.85148148
  92.63529412  87.42041667  88.40142857  91.84875    108.13
  93.61473684  91.64        99.41882353  91.84875    105.95142857
  99.41882353  99.41882353  99.41882353  91.64        87.66478261
  95.98090909  94.85148148  93.61473684 102.175       92.63529412
  89.7573913   88.75166667  94.431875    85.20857143  84.83636364
  91.0404      91.0404     334.91        88.953      105.39
  91.0404      88.953      107.84217391]
[ 82.64 101.69 113.95  89.47  80.   100.34 175.    81.78  89.02 114.92
  95.5  140.56  88.91  77.09  84.43  84.92  88.17 115.93  74.96  79.71
  84.84  75.5   82.89  89.24 126.96 104.48 112.68 110.45  77.24 101.28
  92.18  94.2   81.27 117.24  81.21  80.26  97.75  80.01 147.64 244.96
  86.6  115.3   82.17]


/var/folders/p0/rxjj0qbs28gc1tq82964w38w0000gp/T/ipykernel_23313/3332540717.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_df = res_df.append({'dataset': dataset, 'k': k, 'model': res[0], 'rmse_test': res[1], 'r2_test': res[2], 'mae_test': res[3], 'mse_test': res[4], 'clark_test_A': res[5][0], 'clark_test_B' : res[5][1], 'rmse_val': res[6], 'r2_val': res[7], 'mae_val': res[8], 'mse_val': res[9], 'clark_val_A': res[10][0], 'clark_val_B' : res[10][1]}, ignore_index=True)
/var/folders/p0/rxjj0qbs28gc1tq82964w38w0000gp/T/ipykernel_23313/3332540717.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_df = res_df.append({'dataset': dataset, 'k': k, 'model': res[0], 'rmse_test': res[1], 'r2_test': res[2], 'mae_test': res[3], 'mse_test': res[4], 'clark_test_A': res[5][0], 'clark_test_B' : res[5][1], 'rmse

...

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/var/folders/p0/rxjj0qbs28gc1tq82964w38w0000gp/T/ipykernel_23313/3332540717.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_df = res_df.append({'dataset': dataset, 'k': k, 'model': res[0], 'rmse_test': res[1], 'r2_test': res[2], 'mae_test': res[3], 'mse_test': res[4], 'clark_test_A': res[5][0], 'clark_test_B' : res[5][1], 'rmse_val': res[6], 'r2_val': res[7], 'mae_val': res[8], 'mse_val': res[9], 'clark_val_A': res[10][0], 'clark_val_B' : res[10][1]}, ignore_index=True)
/var/folders/p0/rxjj0qbs28gc1tq82964w38w0000gp/T/ipykernel_23313/3332540717.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

Ada Boost
[ 88.605       86.          89.41352941  84.55        84.77857143
  91.72        99.12142857  93.41411765  80.83818182 104.98
  88.17        89.34181818  88.17        83.036       85.57904762
  95.11214286  95.11214286  85.188       85.57904762  82.82
  98.84538462  97.835       89.62857143  82.83571429  93.41411765
  84.2175      89.34181818  93.41411765  93.36454545  88.79
  85.57904762  84.55        91.10692308  90.765       84.55
  98.84538462  88.17        94.99777778  81.27461538  82.62571429
  91.10692308  87.86        87.86      ]
[ 99.68  95.42  84.43  84.84  87.99  95.58  90.92  83.51  89.96 114.92
 124.24 244.96  94.2  114.71 126.96 108.17  81.13 109.52 117.24 100.34
  89.24 147.64  89.02 226.55 101.28 107.62  98.52 340.14 134.05  92.41
  72.1   88.19  82.89  81.27  77.09 118.61  84.03  92.03  85.62  75.83
 102.03 402.99 104.48]


/var/folders/p0/rxjj0qbs28gc1tq82964w38w0000gp/T/ipykernel_23313/3332540717.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_df = res_df.append({'dataset': dataset, 'k': k, 'model': res[0], 'rmse_test': res[1], 'r2_test': res[2], 'mae_test': res[3], 'mse_test': res[4], 'clark_test_A': res[5][0], 'clark_test_B' : res[5][1], 'rmse_val': res[6], 'r2_val': res[7], 'mae_val': res[8], 'mse_val': res[9], 'clark_val_A': res[10][0], 'clark_val_B' : res[10][1]}, ignore_index=True)
/var/folders/p0/rxjj0qbs28gc1tq82964w38w0000gp/T/ipykernel_23313/3332540717.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_df = res_df.append({'dataset': dataset, 'k': k, 'model': res[0], 'rmse_test': res[1], 'r2_test': res[2], 'mae_test': res[3], 'mse_test': res[4], 'clark_test_A': res[5][0], 'clark_test_B' : res[5][1], 'rmse

Ada Boost
[ 92.75666667  92.18        91.82789474  84.462       89.84272727
  89.16        89.47181818  87.29571429  92.33333333  87.29571429
  87.02692308  89.16        92.161       96.995       93.82590909
  95.436      107.5285      86.572       82.06785714  97.721
 107.5285      97.61466667  94.03941176  99.981      104.06736842
  88.67428571  93.82590909 101.44        95.101       88.14625
  93.92666667  85.46125     88.18272727  91.82789474  88.67428571
 100.86666667  88.67428571 102.40454545  87.17538462  96.79571429
 107.115       93.92666667  95.436     ]
[175.   114.71  94.2   88.91  79.5   80.64 101.28  72.1  124.24  84.8
 104.48  77.24 340.14  89.02 115.3  118.61 144.85  82.6   95.58  90.92
  97.75  89.24  80.4  334.91  74.96 100.31  88.19  88.17 140.56 102.03
 134.05  70.41  83.65  81.21 102.38 105.34  85.62 126.96  97.96  91.14
 114.92 112.42  84.03]


/var/folders/p0/rxjj0qbs28gc1tq82964w38w0000gp/T/ipykernel_23313/3332540717.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_df = res_df.append({'dataset': dataset, 'k': k, 'model': res[0], 'rmse_test': res[1], 'r2_test': res[2], 'mae_test': res[3], 'mse_test': res[4], 'clark_test_A': res[5][0], 'clark_test_B' : res[5][1], 'rmse_val': res[6], 'r2_val': res[7], 'mae_val': res[8], 'mse_val': res[9], 'clark_val_A': res[10][0], 'clark_val_B' : res[10][1]}, ignore_index=True)
/var/folders/p0/rxjj0qbs28gc1tq82964w38w0000gp/T/ipykernel_23313/3332540717.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res_df = res_df.append({'dataset': dataset, 'k': k, 'model': res[0], 'rmse_test': res[1], 'r2_test': res[2], 'mae_test': res[3], 'mse_test': res[4], 'clark_test_A': res[5][0], 'clark_test_B' : res[5][1], 'rmse

In [34]:
res_df.to_csv('results_5.csv', index=False)

In [35]:
features_df

,dataset,k,features,scores
0,/Users/sahilsahu/Desktop/folders/Feature-Extra...,5,"Index(['Tdp_mean', 'Tsys/Tdia_mean', 'Tdw90/Ts...","[0.28407481777879307, 0.23701102096139426, 0.2..."
1,/Users/sahilsahu/Desktop/folders/Feature-Extra...,6,"Index(['Tdw90_median', 'Tpw90_kurtosis', 'IPR_...","[0.2154546104852777, 0.2323123756784593, 0.203..."
2,/Users/sahilsahu/Desktop/folders/Feature-Extra...,7,"Index(['Tdp_mad', 'Tdw25_percentile_75', 'Adp_...","[0.2072288776917941, 0.24564701593260718, 0.22..."
3,/Users/sahilsahu/Desktop/folders/Feature-Extra...,8,"Index(['Tsp_mean', 'Tsp_percentile_25', 'Tsw10...","[0.29687712259659405, 0.2822723516345351, 0.30..."
4,/Users/sahilsahu/Desktop/folders/Feature-Extra...,9,"Index(['Tdw90_percentile_25', 'Tpw10_percentil...","[0.20078386407502924, 0.16106040773882668, 0.2..."
5,/Users/sahilsahu/Desktop/folders/Feature-Extra...,10,"Index(['Tpi_median', 'Tpi_percentile_25', 'Tpp...","[0.30507078497548834, 0.3210168322409799, 0.28..."
6,/Users/sahilsahu/Desktop/folders/Feature-Extra...,11,"Index(['Tdw10_percentile_75', 'Tdw25_percentil...","[0.21012040735950555, 0.18717708859890614, 0.1..."
7,/Users/sahilsahu/Desktop/folders/Feature-Extra...,12,"Index(['Tpi_median', 'Tpi_percentile_25', 'Tpp...","[0.2575485398026074, 0.23930318289424646, 0.27..."
8,/Users/sahilsahu/Desktop/folders/Feature-Extra...,13,"Index(['Tsys_median', 'Tsys_percentile_25', 'T...","[0.1658942885909367, 0.23899068408457147, 0.22..."
9,/Users/sahilsahu/Desktop/folders/Feature-Extra...,14,"Index(['Tsp_mean', 'deltaT_percentile_75', 'Ts...","[0.19720673806817457, 0.38656414027109776, 0.2..."


In [36]:
features_df.to_csv('selected_features_mreg.csv', index=False)